In [ ]:
library(stringr)
library(RCurl)
##HOUSE VOTES DATASET####
votesfile <- getURL("http://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data)
votesdata <- read.table(text=votesfile)
votesdata$V1 <- as.character(votesdata$V1)
splitdat = do.call("rbind",strsplit(votesdata$V1,","))
splitdat[splitdat == "?"] = NA #convert "?" to NA
splitdat = as.data.frame(splitdat)
#splitdat = data.frame(apply(splitdat,2,factor))
str(splitdat)
splitdat = splitdat[complete.cases(splitdat),]
##IRIS DATASET####
irisfile <- getURL("http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data")
Iris <- read.csv(text = irisfile)
colnames(Iris) <- c("sepal.length","sepal.width","petal.length","petal.width","class")
#str(Iris)
Iris.features <- Iris
Iris.features$Iris.setosa <- NULL
#View(Iris.features)
###Determine number of clusters#####
#install.packages("factoextra")
#if(!require(devtools)) install.packages("devtools")
#devtools::install_github("kassambara/factoextra")
#pkgs <- c("cluster",  "NbClust")
#install.packages(pkgs)
library(factoextra)
library(cluster)
library(NbClust)
A <- model.matrix(V1~.,splitdat)
A <- as.data.frame(A)
A <- A[complete.cases(A),]
data <- A
#head(A)
data <- A
#head(data)
data3 <- data
data3$`(Intercept)` <- NULL
data4 <- data3
data4 <- scale(data4) #scale house votes for kmeans++
#scale data
iris.scaled <- scale(Iris[,-5])
#head(iris.scaled)

##K-MEANS++####
kmeansp2 <- function(x, k, iter.max = 10, nstart = 1, ...) {
  n <- nrow(x) # number of data points
  centers <- numeric(k) # IDs of centers
  distances <- matrix(numeric(n * (k - 1)), ncol = k - 1) # distances[i, j]: The distance between x[i,] and x[centers[j],]
  res.best <- list(tot.withinss = Inf) # the best result among <nstart> iterations
  for (rep in 1:nstart) {
    pr <- rep(1, n) # probability for sampling centers
    for (i in 1:(k - 1)) {
      centers[i] <- sample.int(n, 1, prob = pr) # Pick up the ith center
      distances[, i] <- colSums((t(x) - x[centers[i], ])^2) # Compute (the square of) distances to the center
      pr <- distances[cbind(1:n, max.col(-distances[, 1:i, drop = FALSE]))] # Compute probaiblity for the next sampling
    }
    centers[k] <- sample.int(n, 1, prob = pr)
    ## Perform k-means with the obtained centers
    res <- kmeans(x, x[centers, ], iter.max = iter.max, nstart = 1, ...)
    res$inicial.centers <- x[centers, ]
    ## Store the best result
    if (res$tot.withinss < res.best$tot.withinss) {
      res.best <- res
    }
  }
  res.best
}

##IRIS K++####
iris.kmeansp2 <- kmeansp2(iris.scaled,k=3)
tab_kmeanssp2 <- table(iris.kmeansp2$cluster,Iris$class)
#tab_kmeanssp2
acc_iris <- sum(tab_kmeansp2[1,1],tab_kmeansp2[2,2],tab_kmeansp2[3,3])/sum(tab_kmeansp2)
print(c("Accuracy IRIS K-means++: ",acc_iris))
##HOUSE VOTES K++####
hv.kmeansp2 <- kmeansp2(data4,k=2)
tab_kmeansp2.hv <- table(hv.kmeansp2$cluster,splitdat$V1)
#tab_kmeansp2.hv
acc_hv <- sum(tab_kmeansp2.hv[1,1],tab_kmeansp2.hv[2,2])/sum(tab_kmeansp2.hv)
print(c("Accuracy HOUSE VOTES K-means++: ",acc_hv))




In [ ]:
##Elbow method IRIS K-Means####
set.seed(123)
#Compute and plot wss for k = 2 to k = 15
k.max <- 15 # Maximal number of clusters
data.scaled <- iris.scaled
wss <- sapply(1:k.max, 
              function(k){kmeans(data.scaled, k, nstart=10 )$tot.withinss})
plot(1:k.max, wss,
     type="b", pch = 19, frame = FALSE, 
     xlab="Number of clusters K",
     ylab="Total within-clusters sum of squares", main = "Elbow Plot IRIS")
#Elbow line indicating 3 is a good cluster number
abline(v = 3, lty =2)

##Elbow method HOUSE VOTES K-Means####
#created dummy variables for categorical predictors
A <- model.matrix(V1~.,splitdat)
A <- as.data.frame(A)
A <- A[complete.cases(A),]
data <- A
wss <- sapply(1:k.max, 
              function(k){kmeans(data, k, nstart=10 )$tot.withinss})
plot(1:k.max, wss,
     type="b", pch = 19, frame = FALSE, 
     xlab="Number of clusters K",
     ylab="Total within-clusters sum of squares",main = "Elbow Plot HOUSE VOTES")
#Elbow line indicating 2 is a good cluster number
abline(v = 2, lty =2)

##K-MEANS IRIS#######

#k = 3 from Elbow method above
Iris.features$class <- NULL
results <- kmeans(Iris.features, 3)
#results
tab_iris_kmeans <- table(Iris$class,results$cluster)
acc_iris_km <- sum(tab_iris_kmeans[1,1],tab_iris_kmeans[2,2],tab_iris_kmeans[3,3])/sum(tab_iris_kmeans)
print(c("Accuracy Iris K-Means: ",acc_iris_km))
#str(Iris)
#Sepal length and width cluster
plot(Iris[c("sepal.length","sepal.width")],col=results$cluster,main = "Predicted")
plot(Iris[c("sepal.length","sepal.width")],col = Iris$class,main = "Actual")
#Petal length and width cluster
plot(Iris[c("petal.length","petal.width")],col=results$cluster,main = "Predicted")
plot(Iris[c("petal.length","petal.width")],col=Iris$class,main = "Actual")

##K-MEANS HOUSE VOTES#######
head(splitdat)
#created dummy variables for categorical predictors
A <- model.matrix(V1~.,splitdat)
head(A)
#k = 2 from Elbow method above
res_hv <- kmeans(A,2)
#res_hv
tab_hv_kmeans <- table(splitdat$V1,res_hv$cluster)
#tab_hv_kmeans
acc_hv_kmeans <- sum(tab_hv_kmeans[1,1],tab_hv_kmeans[2,2])/sum(tab_hv_kmeans)
print(c("Accuracy House Votes K-Means: ",acc_hv_kmeans))
#plot(splitdat[c("V2","V3")],col=res_hv$cluster,main = "Predicted")

In [ ]:
##MIXTURE MODELS IRIS####
library(mclust)
#xyclust <- Mclust(Iris.features) #Just two clusters with BIC, not a good choice
xyclust <- Mclust(Iris.features,G=3)
#plot(xyclust)
#summary(xyclust)
#xyclust$classification
tab_mclust_iris <- table(Iris$class,xyclust$classification)
acc_iris_mm <- sum(tab_mclust_iris[1,1],tab_mclust_iris[2,2],tab_mclust_iris[3,3])/sum(tab_mclust_iris)
print(c("Accuracy Iris Mixture Models: ",acc_iris_mm))

##MIXTURE MODELS HOUSE VOTES####
head(A)
data <- A
data <- as.data.frame(data)
data$`(Intercept)` <- NULL
#xyclust.hv <- Mclust(data)# Considers 5 clusters, not a good choice
xyclust.hv <- Mclust(data,G=2)
#plot(xyclust.hv)
tab_mclust_hv <- table(xyclust.hv$classification,splitdat$V1)
acc_hv_mm <- sum(tab_mclust_hv[1,1],tab_mclust_hv[2,2])/sum(tab_mclust_hv)
print(c("Accuracy House Votes Mixture Models: ",acc_hv_mm))

In [ ]:
##HIERARCHICAL CLUSTERING IRIS####
fn <- fviz_nbclust(iris.scaled,pam,method = "wss")
#fn
#we can see an elbow at k = 3, k=2 could also be considered but the slope is steeper for 2-3 than 3-15 so we choose 3 as elbow point
fn+geom_vline(xintercept = 3, linetype = 2)
dist.iris <- dist(as.matrix(Iris))
hc <- hclust(dist.iris)
clustCut <- cutree(hc,3)
tab_iris_clust <- table(clustCut,Iris$class)
#tab_iris_clust
acc_iris_hc <- sum(tab_iris_clust[1,1],tab_iris_clust[2,2],tab_iris_clust[3,3])/sum(tab_iris_clust)
#plot(hc)
print(c("Accuracy Iris Hierarchical Cluster:",acc_iris_hc))

##HIERARCHICAL CLUSTERING HOUSE VOTES#####
fn2 <- fviz_nbclust(data,pam,method = "wss")
#fn2
#we can see a clear elbow at k = 2
fn2+geom_vline(xintercept = 2, linetype = 2)
dist.hv <- dist(as.matrix(A))
hc.hv <- hclust(dist.hv)
#plot(hc.hv)
clustCut.hv <- cutree(hc.hv,2)
tab_hv_clust <- table(clustCut.hv,splitdat$V1)
#tab_hv_clust
acc_hv_hc <- sum(tab_hv_clust[1,1],tab_hv_clust[2,2])/sum(tab_hv_clust)
print(c("Accuracy House Votes Hierarchical Cluster: ",acc_hv_hc))


In [ ]:
##VISUALIZATION PCA-IRIS####
log.ir <- log(Iris[,1:4])
ir.species <- Iris[,5]
#Apply PCA, scaled
ir.pca <- prcomp(log.ir,center = TRUE,scale. = TRUE)
#ir.pca
plot(ir.pca,type="l")
library(devtools)
#install_github("ggbiplot", "vqv")
library(ggbiplot)
g <- ggbiplot(ir.pca, obs.scale = 1, var.scale = 1, 
              groups = ir.species, ellipse = TRUE, 
              circle = TRUE)
g <- g + scale_color_discrete(name = '')
g <- g + theme(legend.direction = 'horizontal', 
               legend.position = 'top')
print(g)

##VISUALIZATION PCA-HOUSE VOTES####
data3 <- data
data3$`(Intercept)` <- NULL
hv.class <- splitdat$V1
hv.pca <- prcomp(data3,center = TRUE)
hv.pca
plot(hv.pca,type="l")
g2 <- ggbiplot(hv.pca,groups=hv.class,ellipse = TRUE, 
               circle = TRUE)
g2 <- g2 + scale_color_discrete(name = '')
g2 <- g2 + theme(legend.direction = 'horizontal', 
               legend.position = 'top')
print(g2)

##VISUALIZATION IRIS AUTOENCODER####
#install.packages("h2o")
library(h2o)
h2o.init()
head(Iris)
str(Iris)
iris.hex <- as.h2o(Iris)
iris.dl <- h2o.deeplearning(x=1:4,y=5,training_frame=iris.hex,hidden=c(10,5,10))
#extract features from middle layers
#layer 2
mid.features <- h2o.deepfeatures(iris.dl,iris.hex,layer = 2)
plotdata <- as.data.frame(mid.features)
plotdata$label <- as.character(as.vector(Iris$class))
qplot(plotdata$DF.L2.C1,plotdata$DF.L2.C2,data=plotdata,color=label)
#layer 3
mid.features3 <- h2o.deepfeatures(iris.dl,iris.hex,layer = 3)
plotdata3 <- as.data.frame(mid.features3)
plotdata3$label <- as.character(as.vector(Iris$class))
qplot(plotdata3$DF.L3.C1,plotdata3$DF.L3.C2,data=plotdata3,color=label,main = "Hidden layer 3 neural net")
#prediction
predictions <- h2o.predict(iris.dl,iris.hex)
#predictions


##VISUALIZATION House Votes AUTOENCODER####
h2o.shutdown()
#disabling assertions in H2O
h2o.init(nthreads=-1,enable_assertions = FALSE)
nrow(A)
nrow(splitdat)
nrow(data)
data <- as.data.frame(scale(A))
head(data)
data$V1 <- as.character(as.vector(splitdat$V1))
data$V1 <- as.factor(data$V1)
head(data)
data2 <- data
data2$`(Intercept)`<- NULL
head(data2)
hv.hex <- as.h2o(data2)
hv.dl <- h2o.deeplearning(x=1:16,y=17,training_frame=hv.hex,hidden=c(20,10,2,10,20))
hv.features <- h2o.deepfeatures(hv.dl,hv.hex,layer=3)
plotdata3.hv <- as.data.frame(hv.features)
plotdata3.hv$label <- as.character(as.vector(splitdat$V1))
qplot(plotdata3.hv$DF.L3.C1,plotdata3.hv$DF.L3.C2,data=plotdata3.hv,color=label,main="Hidden layer 3 Neural Net")
